In [1]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import networkx as nx
import numpy as np
import pickle

/var/folders/lk/t243gzpx379bzd9mvx8dcsmm0000gp/T/ipykernel_55367/1766039503.py:1: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd


In [2]:
# 파일 불러오기
with open('../../data/df.pkl', 'rb') as f:
    df = pickle.load(f)


In [3]:
# 파일 불러오기
with open('../../data/df_qna.pkl', 'rb') as f:
    df_qna = pickle.load(f)

In [4]:
# 파일 불러오기
with open('../../data/node.pkl', 'rb') as f:
    node = pickle.load(f)

In [5]:
# 파일 불러오기
with open('../../data/edgelist.pkl', 'rb') as f:
    edgelist = pickle.load(f)

In [6]:
# 파일 불러오기
centernode = pd.read_csv('../../data/centernode_gephi.csv')

In [7]:
print(df['q_owneruserid'].min())
print(df['q_owneruserid'].max())

214
23031022


In [8]:
centernode

,Source,Target,Type,Id,Label,Weight
0,22801094,1180351,Directed,12212,NaN,1
1,11143113,1180351,Directed,12040,NaN,1
2,20677096,1180351,Directed,12009,NaN,1
3,2449161,1180351,Directed,11354,NaN,1
4,14701970,1180351,Directed,11222,NaN,1
...,...,...,...,...,...,...
8587,1705983,1585136,Directed,3008,NaN,1
8588,8002175,8002175,Directed,4810,NaN,1
8589,1045142,2083900,Directed,3977,NaN,1
8590,8002175,1045142,Directed,9755,NaN,1


Data wrangling

In [9]:
print("전체 edge 개수 : ", df.shape[0])
print("중심부 edge 개수 : ", centernode.shape[0])

전체 edge 개수 :  12428
중심부 edge 개수 :  8592


In [10]:
print("전체 node 개수 : ", len(df_qna['user_id'].unique()))
print("중심부 node 개수 : ", len(np.union1d(centernode['Source'].values, centernode['Target'].values)))

전체 node 개수 :  11392
중심부 node 개수 :  6681


In [11]:
print("데이터 셋 내 최초 질문 일시 : ", df['q_creationdate'].min())
print("데이터 셋 내 최종 질문 일시 : ", df['q_creationdate'].max())

데이터 셋 내 최초 질문 일시 :  2023-08-03 01:04:49.693000
데이터 셋 내 최종 질문 일시 :  2023-12-03 08:29:11.803000


In [12]:
print("데이터 셋 내 최초 답변 일시 : ", df['a_creationdate'].min())
print("데이터 셋 내 최종 답변 일시 : ", df['a_creationdate'].max())

데이터 셋 내 최초 답변 일시 :  2023-08-03 01:23:36.673000
데이터 셋 내 최종 답변 일시 :  2023-12-03 10:02:09.807000


In [14]:
# centernode_list = np.union1d(centernode['Source'].values, centernode['Target'].values)
# centernode_list = [node[x] for x in centernode_list]

In [ ]:
# centernode['Source_org'] = centernode['Source'].apply(lambda x : node[x])
# centernode['Target_org'] = centernode['Target'].apply(lambda x : node[x])

In [ ]:
# # centernode[['Source_org', 'Target_org']]
# # centernode[['Source', 'Target']]
# # .drop_duplicates()

# centernode[['Source_org', 'Target_org']].drop_duplicates()

In [ ]:
tmp = pd.merge(df, centernode, left_on = ['q_owneruserid', 'a_owneruserid'], right_on = ['Source_org', 'Target_org'])

In [ ]:
tmp[['q_owneruserid', 'a_owneruserid']]

In [ ]:
tmp[tmp[['q_owneruserid', 'a_owneruserid']].duplicated()]

In [ ]:
q_df = df[['q_id', 'q_creationdate']].copy()
q_df.drop_duplicates(inplace=True)

a_df = df[['a_id', 'a_creationdate']].copy()
a_df.drop_duplicates(inplace=True)

In [ ]:
q_df['create_month'] = pd.to_datetime(q_df['q_creationdate']).dt.month
a_df['create_month'] = pd.to_datetime(a_df['a_creationdate']).dt.month

In [ ]:
q_df = q_df.groupby(['create_month']).count()['q_id'].reset_index()
a_df = a_df.groupby(['create_month']).count()['a_id'].reset_index()

In [ ]:
qna_month = pd.merge(q_df,a_df, on  = 'create_month')

In [ ]:
qna_month

In [ ]:
idx = np.arange(qna_month.shape[0])
w = 0.15

plt.figure(figsize = (10, 5))
plt.title('user count who leaves question and answer per month ')
plt.xlabel('month(in 2023)')
plt.ylabel('user count')
plt.bar(idx - w * w, qna_month['q_id'], width = w, label = 'question count')
plt.bar(idx + w, qna_month['a_id'], width = w, label = 'answer count')
plt.xticks(idx, qna_month['create_month'], rotation = 30)
plt.legend(ncol = 1)
plt.show()

In [ ]:
idx = np.arange(qna_month.shape[0])
w = 0.15

plt.figure(figsize = (10, 5))
plt.title('user count who leaves question and answer per month ')
plt.xlabel('month(in 2023)')
plt.ylabel('user count')
plt.bar(idx - w * w, qna_month['q_id'], width = w, label = 'question count')
plt.bar(idx + w, qna_month['a_id'], width = w, label = 'answer count')
plt.xticks(idx, qna_month['create_month'], rotation = 30)
plt.legend(ncol = 1)
plt.show()

## 유저 별 질문/답변 횟수를 히스토그램으로 출력 
* 대부분의 유저들이 질문과 답변을 안한다는 것을 알 수 있음 

In [ ]:
fig, ax = plt.subplots()

ax.hist([df_qna['question_cnt'], df_qna['answer_cnt']], # specify data
        bins = 20, # number of bins
        color = ['green', 'deeppink'], # bar color
        histtype = 'step', # type of histogram
        label = ['question', 'answer']) # create legened labels

ax.set_title('Histogram of answer or question count', fontsize = 15) # title
ax.set_xlabel('count', fontsize = 12.5) # x-axis label
ax.set_ylabel('value', fontsize = 12.5) # y-axis label

ax.legend(loc = 'upper right', fontsize = 10) # add legend

ax.grid(True, linestyle = ':', linewidth = 0.5) # grid

plt.show();

## 같은 데이터를 활용하여 누적분포 함수 시각화
* 질문횟수와 답변횟수가 비슷한 기울기를 보이다가, 질문횟수가 답변횟수보다 먼저 1로 수렴 
* 답변횟수보다 질문횟수가 0에 더 가깝다는것을 알 수 있음

In [ ]:
### cdf plot for multiple groups

fig, ax = plt.subplots()

sns.ecdfplot(x = df_qna['question_cnt'],
             color = 'green',
             label = 'question',
             ax = ax)
sns.ecdfplot(x = df_qna['answer_cnt'],
             color = 'deeppink',
             label = 'answer',
             ax = ax)

ax.set_title('Number of question or answer count', fontsize=15)
ax.set_xlabel('count', fontsize = 12.5)
ax.set_ylabel('CDF', fontsize = 12.5)

ax.legend(loc = 'upper right', fontsize = 10)

ax.grid(True, linestyle=':', linewidth = 0.5)

plt.show();

## 유저 별 답변비율 시각화
* 위의 그래프를 보면 대부분의 유저들이 질문도 답볃도 하지 않는 상황
* 그럼 유저별로 질문 답변 횟수 사이에 차이가 존재하는지 비율로 확인 
* (유저별 질문횟수)/(유저별 질문횟수+유저별 답변횟수)
* 대부분의 유저들이 거의 답변을 하지 않고, 몇몇의 유저들이 답변만 하는 것을 알 수 있음

In [ ]:
df_qna['question_rate'] = np.round(df_qna['question_cnt']/(df_qna['question_cnt']+df_qna['answer_cnt'])*100, 2)

In [ ]:
df_qna['answer_rate'] = 100-df_qna['question_rate'] 

In [ ]:
fig, axs = plt.subplots(1, 1, figsize = (10, 5))

axs.hist(df_qna['answer_rate'] , # plot the original histogram
         bins = 50,
         color = 'skyblue',
         edgecolor = 'black')
axs.set_title('Histogram of answer_rate', fontsize = 15)
axs.set_xlabel('Question rate per user', fontsize = 15)
axs.set_ylabel('Number of Users', fontsize = 15)



In [ ]:
df_qna

In [ ]:
fig_scatter_reg, ax = plt.subplots(figsize = (10, 6))

ax.scatter(df_qna['question_cnt'], df_qna['answer_cnt'], color = 'purple', alpha = 0.6)

ax.set_title('Question and Answer')
ax.set_xlabel('Question')
ax.set_ylabel('Answer')


plt.show();

## 질문 혹은 답변 횟수가 0인 유저 별 비율 시각화
* 그래프를 보면 어떤 수치든 양극단의 비율이 높음(정규분포를 띄지 않음)
* 극단에 존재하는 유저(답변횟수가 0 이거나 질문 횟수가 0)들을 대상으로 시각화 진행
* 한번도 질문하지 않은 유저는 10706명, 한번도 답변하지 않은 유저는 23442명으로 답하지 않은 유저가 약 2.18 배 많음 
* 한번도 답변하지 않은 유저의 경우 모두 25회 미만으로 질문하였지만, 한번도 대답하지 않은 유저의 경우 분산이 크며 경우에 따라 200회 이상 답변한 유저도 존재 
* 답변하는 유저가 더 적은 수지만 훨씬 더 활발하게 활동한다는것을 알 수 있음 

In [ ]:
zero_q_cnt = df_qna.loc[df_qna['question_cnt']==0, 'answer_cnt']
zero_a_cnt = df_qna.loc[df_qna['answer_cnt']==0, 'question_cnt']

fig,ax = plt.subplots()
ax.boxplot([zero_q_cnt,zero_a_cnt])
# plt.title('Species_bill_length')
plt.xticks([1,2],['zero question user\'s \nanswer count', 'zero answer user\'s \nquestion count'])
plt.show();

In [ ]:
print(len(zero_q_cnt))
print(len(zero_a_cnt))
print(len(zero_a_cnt)/len(zero_q_cnt))

In [ ]:
q_reputation = df[['q_owneruserid','q_reputation']].rename(columns = {'q_owneruserid':'user_id', 'q_reputation':'reputation'})
a_reputation = df[['a_owneruserid','a_reputation']].rename(columns = {'a_owneruserid':'user_id', 'a_reputation':'reputation'})

In [ ]:
df_reputation = pd.concat([q_reputation, a_reputation], axis = 0).drop_duplicates()

In [ ]:
df_qna = pd.merge(df_qna, df_reputation, on = 'user_id', how='left')

In [ ]:
df_qna.isna().sum()

In [ ]:
df_qna.sort_values(by = ['question_cnt'], ascending=False)

In [ ]:
df_qna.sort_values(by = ['answer_rate'], ascending=False)

In [ ]:
zero_q_rep = df_qna.loc[df_qna['question_cnt']==0, 'reputation']

In [ ]:
zero_a_rep = df_qna.loc[df_qna['answer_cnt']==0, 'reputation']


## 질문 혹은 답변 횟수가 0인 유저 별 stackoverflow 평판(reputation) 시각화
* reputation이 높아질수록 유저의 수도 적어짐
* 답변하는 유저가 더 적은 수지만 커뮤니티에서 평판을 쌓아간다는 것을 알 수 있음 


In [ ]:
fig,ax = plt.subplots()
ax.boxplot([zero_q_rep,zero_a_rep])
# plt.title('Species_bill_length')
plt.xticks([1,2],['zero question user\'s \nreputation', 'zero answer user\'s \nreputation'])
plt.show();